In [10]:
import numpy as np
import pandas as pd
import scipy.io
import scipy.constants as sc
import matplotlib.pyplot as plt
from pygmid import Lookup as lk
import math
from spyci import spyci

#### Technology data

In [11]:
n = lk('../../lookuptable/skywater130a/nfet_01v8.mat')
p = lk('../../lookuptable/skywater130a/pfet_01v8.mat')

#### Specifications

In [12]:
T = 300; ib = 300e-9; cl = 1e-12
type = ['n', 'p', 'n', 'p', 'n', 'p', 'n']
l = np.array([5, 5,  5, 5, 5, 5, 5])
w = np.array([4, 6, 3, 27, 4.5, 11.5, 3])
nf = np.array([1, 1, 1, 1, 1, 1, 1])

In [14]:
id = np.array([ib/2, ib/2, ib, 2*ib, 2*ib, ib, ib])
gm_id=np.zeros(len(l))
gm_cgg=np.zeros(len(l))
gm_cdd=np.zeros(len(l))
gm_cgd=np.zeros(len(l))
gm_gds=np.zeros(len(l))
sth_gm=np.zeros(len(l))

for i in range(len(l)):
    if type[i] == 'n':
      gm_id[i] = n.lookup('GM_ID', ID_W=id[i]/w[i], L=l[i], VDS=0.3)
      gm_cgg[i] = n.lookup('GM_CGG', ID_W=id[i]/w[i], L=l[i], VDS=0.3)
      gm_cdd[i] = n.lookup('GM_CDD', ID_W=id[i]/w[i], L=l[i], VDS=0.3)
      gm_cgd[i] = n.lookup('GM_CGD', ID_W=id[i]/w[i], L=l[i], VDS=0.3)
      gm_gds[i] = n.lookup('GM_GDS', ID_W=id[i]/w[i], L=l[i], VDS=0.3)
      sth_gm[i] = n.lookup('STH_GM', ID_W=id[i]/w[i], L=l[i], VDS=0.3)
    else:
      gm_id[i] = p.lookup('GM_ID', ID_W=id[i]/w[i], L=l[i], VDS=0.3) 
      gm_cgg[i] = p.lookup('GM_CGG', ID_W=id[i]/w[i], L=l[i], VDS=0.3)
      gm_cdd[i] = p.lookup('GM_CDD', ID_W=id[i]/w[i], L=l[i], VDS=0.3)
      gm_cgd[i] = p.lookup('GM_CGD', ID_W=id[i]/w[i], L=l[i], VDS=0.3)
      gm_gds[i] = p.lookup('GM_GDS', ID_W=id[i]/w[i], L=l[i], VDS=0.3)
      sth_gm[i] = p.lookup('STH_GM', ID_W=id[i]/w[i], L=l[i], VDS=0.3)

gm = id*gm_id
gds = gm/gm_gds
cgg = gm/gm_cgg
cdd = gm/gm_cgg
cgd = gm/gm_cgd
gamma = sth_gm/(4*sc.Boltzmann*T)

df = pd.DataFrame( [gm_id, gm_cgg/1e9/2/np.pi, gm_gds, gm/1e-6, gds/1e-6, cgg/1e-15, cdd/1e-15, cgd/1e-15, gamma], \
                   ['gm_id (S/A)', 'ft (GHz)', 'gm_gds', 'gm (uS)', 'gds (uS)', 'cgg (fF)', 'cdd (fF)', 'cgd (fF)', 'gamma'], columns=['M1', 'M3', 'M5', 'M7', 'M8', 'M10', 'M6']); df.round(2)

,M1,M3,M5,M7,M8,M10,M6
gm_id (S/A),22.26,16.60,19.92,16.75,18.93,16.53,19.92
ft (GHz),0.01,0.00,0.01,0.00,0.02,0.00,0.01
gm_gds,210.11,657.49,221.79,673.26,220.02,651.33,221.79
gm (uS),3.34,2.49,5.98,10.05,11.36,4.96,5.98
gds (uS),0.02,0.00,0.03,0.01,0.05,0.01,0.03
cgg (fF),95.74,120.55,76.13,535.22,115.94,232.16,76.13
cdd (fF),95.74,120.55,76.13,535.22,115.94,232.16,76.13
cgd (fF),1.65,0.98,1.52,4.22,2.46,1.91,1.52
gamma,1.53,0.66,1.80,0.64,1.88,0.67,1.80


In [18]:

# A0
A0 = gm[0]/(gds[0]+gds[1])*gm[3]/(gds[3]+gds[4])
A0 = 20*np.log10(A0)
# # UGF
# cltot = cl + cdd[1] + cdd[2]
# UGF = gm[1]/cltot/2/np.pi

# # PM
# fp2 = gm_cgg[2]/2/np.pi
# phip2 = -np.arctan(UGF/fp2)*180/np.pi
# fz2 = 2*fp2
# phiz2 = +np.arctan(UGF/fz2)*180/np.pi
# fz3 = gm[1]/cgd[1]/2/np.pi
# phiz3 = -np.arctan(UGF/fz3)*180/np.pi
# PM= 90 +phip2 +phiz2 +phiz3 

# # NOI
# NOI = ( gamma[1]*(1+2*gamma[2]/gamma[1]) * sc.Boltzmann*T/cltot)**0.5

df = pd.DataFrame( [A0], \
                   ['A0'], columns=['Value']); df.round(2)
# df = pd.DataFrame( [A0, UGF/1e6, PM, NOI/1e-6], \
#                    ['A0', 'UGF (MHz)', 'PM (deg)', 'Noise (uVrms)'], columns=['Value']); df.round(2)

,Value
A0,88.17


In [ ]:
# we calculate the dc gain
gm_gds_m12 = n.lookup('GM_GDS', GM_ID=gm_id_m12, L=l_12, VDS=0.3, VSB=0)
gm_gds_m34 = p.lookup('GM_GDS', GM_ID=gm_id_m34, L=l_34, VDS=0.3, VSB=0)
gm_gds_m7 = p.lookup('GM_GDS', GM_ID=gm_id_m7, L=l_7, VDS=0.3, VSB=0)
gm_gds_m8 = n.lookup('GM_GDS', GM_ID=gm_id_m8, L=l_8, VDS=0.3, VSB=0)

gds_m12 = gm_m12 / gm_gds_m12
gm_m34 = gm_id_m34 * id_m12
gds_m34 = gm_m34 / gm_gds_m34

gds_m7 = gm_m7 / gm_gds_m7
gm_m8 = gm_id_m8 * id_m7
gds_m8 = gm_m8 / gm_gds_m8

RI = 1/(gds_m12 + gds_m34)
RII = 1/(gds_m7 + gds_m8)
a1 = gm_m12 * RI
a2 = gm_m7  * RII
a0 = a1 * a2
print('a0 =', round(20*np.log10(a0), 1), 'dB')
print('a1 =', round(20*np.log10(a1), 1), 'dB')
print('a2 =', round(20*np.log10(a2), 1), 'dB')

In [ ]:
# Parasitic caps at 1st stage output (Miller node)
gm_cgd_m12 = n.lookup('GM_CGD', GM_ID=gm_id_m12, L=l_12, VDS=0.9, VSB=0)
gm_cdd_m12 = n.lookup('GM_CDD', GM_ID=gm_id_m12, L=l_12, VDS=0.9, VSB=0)
gm_cgd_m7  = p.lookup('GM_CGD', GM_ID=gm_id_m7,  L=l_7,  VDS=0.9, VSB=0)

C1 = abs(gm_m12/gm_cgd_m12) + abs(gm_m12/gm_cdd_m12) + abs(gm_m7/gm_cgd_m7)

# Parasitics at output node (2nd stage output)
gm_cdd_m7 = p.lookup('GM_CDD', GM_ID=gm_id_m7, L=l_7, VDS=0.9, VSB=0)
gm_cdd_m8 = n.lookup('GM_CDD', GM_ID=gm_id_m8, L=l_8, VDS=0.9, VSB=0)

C2 = abs(gm_m7/gm_cdd_m7) + abs(gm_m8/gm_cdd_m8)

# Total load including parasitics
C_total_load = c_load + C2
C_total_firstnode = C1  # used with Cc

print("Parasitic @ Miller node (C1):", round(C1/1e-15, 2), "fF")
print("Parasitic @ output node (C2):", round(C2/1e-15, 2), "fF")

# Unity gain frequency approximation including parasitics
f_UGF = gm_m12 / (2*np.pi*(Cc + C1 + C_total_load))

print("Estimated UGF with parasitics =", round(f_UGF/1e6, 2), "MHz")


In [ ]:
# we can now look up the VGS of the MOSFET
vgs_m12 = n.look_upVGS(GM_ID=gm_id_m12, L=l_12, VDS=0.3, VSB=0.0)
vgs_m34 = p.look_upVGS(GM_ID=gm_id_m34, L=l_34, VDS=0.3, VSB=0.0) 
vgs_m56 = n.look_upVGS(GM_ID=gm_id_m56, L=l_56, VDS=0.3, VSB=0.0) 
vgs_m7 = p.look_upVGS(GM_ID=gm_id_m7, L=l_7, VDS=0.3, VSB=0.0) 
vgs_m8 = n.look_upVGS(GM_ID=gm_id_m8, L=l_8, VDS=0.3, VSB=0.0) 

print('vgs_12 =', round(float(vgs_m12), 3), 'V')
print('vgs_34 =', round(float(vgs_m34), 3), 'V')
print('vgs_56 =', round(float(vgs_m56), 3), 'V')
print('vgs_7 =', round(float(vgs_m7), 3), 'V')
print('vgs_8 =', round(float(vgs_m8), 3), 'V')

In [ ]:
# calculate settling time due to slewing with the calculated bias current
t_slew = (c_load + c_load_parasitic) * output_voltage / i_total
print('slewing time =', round(t_slew/1e-6, 3), 'µs')
t_settle = 5/(2*np.pi*f_bw)
print('settling time =', round(t_settle/1e-6, 3), 'µs')

In [ ]:
# calculate voltage gain error
gain_error = a0 / (1 + a0)
print('voltage gain error =', round((gain_error-1)*100, 1), '%')

In [ ]:
# calculate total rms output noise
sth_m12 = n.lookup('STH_GM', VGS=vgs_m12, L=l_12, VDS=0.75, VSB=0) * gm_m12
gamma_m12 = sth_m12/(4*1.38e-23*300*gm_m12)

sth_m34 = p.lookup('STH_GM', VGS=vgs_m34, L=l_34, VDS=0.75, VSB=0) * gm_m34
gamma_m34 = sth_m34/(4*1.38e-23*300*gm_m34)

output_noise_rms = np.sqrt(1.38e-23*300 / (c_load + c_load_parasitic) * (2*gamma_m12 + 2*gamma_m34 * gm_m34/gm_m12))
print('output noise =', round(output_noise_rms/1e-6, 1), 'µVrms')

In [ ]:
# calculate all widths
id_w_m12 = n.lookup('ID_W', GM_ID=gm_id_m12, L=l_12, VDS=0.3, VSB=0)
w_12 = id_m12 / id_w_m12
w_12_round = max(round(w_12*2)/2, 0.5)
print('M1/2 W =', round(w_12, 2), 'um, rounded W =', w_12_round, 'um')

id_m34 = id_m12
id_w_m34 = p.lookup('ID_W', GM_ID=gm_id_m34, L=l_34, VDS=0.3, VSB=0)
w_34 = id_m34 / id_w_m34
w_34_round = max(round(w_34*2)/2, 0.5) 
print('M3/4 W =', round(w_34, 2), 'um, rounded W =', w_34_round, 'um')

id_w_m5 = n.lookup('ID_W', GM_ID=gm_id_m56, L=l_56, VDS=0.3, VSB=0)
w_5 = i_total / id_w_m5
w_5_round = max(round(w_5*2)/2, 0.5)
print('M5 W =', round(w_5, 2), 'um, rounded W =', w_5_round, 'um')
w_6 = w_5_round * i_bias_in / i_total
w_6_round = max(round(w_6*2)/2, 0.5)
print('M6 W =', round(w_6_round, 2), 'um')

id_w_m7 = p.lookup('ID_W', GM_ID=gm_id_m7, L=l_7, VDS=0.3, VSB=0)
w_7 = id_m7 / id_w_m7
w_7_round = max(round(w_7*2)/2, 0.5)
print('M7 W =', round(w_7, 2), 'um, rounded W =', w_7_round, 'um')

id_w_m8 = n.lookup('ID_W', GM_ID=gm_id_m8, L=l_8, VDS=vgs_m8, VSB=0)
w_8 = id_m7 / id_w_m8
w_8_round = max(round(w_8*2)/2, 0.5)
print('M8 W =', round(w_8, 2), 'um, rounded W =', w_8_round, 'um')

In [ ]:
vout_max = 1.6
vout_min = 0.2

# Headroom checks
headroom_M1 = vdd_min - vgs_m34 + vgs_m12 - vin_max
headroom_M4 = vdd_min - vin_max
headroom_M5 = vin_min - vgs_m12
headroom_M7 = vout_max - vgs_m7
headroom_M8 = vdd_min - vout_min - vgs_m8

print(f"Headroom M1 = {headroom_M1:.3f} V")
print(f"Headroom M4 = {headroom_M4:.3f} V")
print(f"Headroom M5 = {headroom_M5:.3f} V")
print(f"Headroom M7 = {headroom_M7:.3f} V")
print(f"Headroom M8 = {headroom_M8:.3f} V")

In [ ]:
# print out final design values
print('5T-OTA dimensioning:')
print('--------------------')
print('M1/2 W=', w_12_round, ', L=', l_12)
print('M3/4 W=', w_34_round, ', L=', l_34)
print('M5   W=', w_5_round, ', L=', l_56)
print('M6   W=', w_6_round, ', L=', l_56)
print()
print('5T-OTA performance summary:')
print('---------------------------')
print('supply current =', round(i_total/1e-6, 1), 'µA')
print('output noise =', round(output_noise_rms/1e-6, 1), 'µVrms')
print('voltage gain error =', round((gain_error-1)*100, 1), '%')
print('unity gain bandwidth incl. parasitics =', round(f_bw/1e6, 2), 'MHz')
print('turn-on time (slewing+settling) =', round((t_slew+t_settle)/1e-6, 3), 'µs')
print()
print('5T-OTA bias point check:')
print('------------------------')
print('headroom M1 =', round(vdd_min-vgs_m34+vgs_m12-vin_max, 3), 'V')
print('headroom M4 =', round(vdd_min-vin_max, 3), 'V')
print('headroom M5 =', round(vin_min-vgs_m12, 3), 'V')